# Лабораторна робота №3

**Тема:** Тренування, оцінювання та оптимізація моделі машинного навчання.

**Мета:** Підібрати відповідну модель для бізнес-проблеми, налаштувати її тренування, обрати метрики, оптимізувати гіперпараметри та оцінити важливість ознак, використовуючи реальні дані з бази підприємства.

## Завдання 1
Виберіть техніку моделювання, яка підходить для вирішення задачі вашого підприємства. Обґрунтуйте, чому саме класифікація, регресія або інший підхід є релевантним до вашої бізнес-проблеми.

Оскільки ми прагнемо передбачити *ймовірність ожиріння*, аналіз *анкетних даних* повинен давати *ймовірність* потрапляння клієнта в одну із наявних категорій ваги, яких у датасеті ми маємо сім:

1. Insufficient_Weight
2. Normal_Weight
3. Overweight_Level_I
4. Overweight_Level_II
5. Obesity_Type_I
6. Obesity_Type_II
7. Obesity_Type_III

Таким чином, наша техніка моделювання зводиться до **мультикласової класифікації**, оскільки маємо більше двох категорій.

## Завдання 2
Опишіть модельне припущення. У звіті до лабораторної поясніть, які припущення робить обрана модель (наприклад, лінійність, незалежність змінних, балансованість класів) і наскільки вони виконуються у ваших даних.

Згідно з результатами лабораторної роботи №2, модель буде робити наступні припущення:

- у датасеті відсутні *пропущенні* значення
- усі атрибути класів є *числовими*
- дані **не** розподілені рівномірно
- аномалії в датасеті *відсутні*

## Завдання 3
Оберіть 2–3 релевантні метрики для оцінки моделі. Наприклад: accuracy, F1-score, recall, precision, MAE, RMSE — залежно від типу задачі. Обґрунтуйте, чому саме ці метрики відповідають вашій меті (наприклад, у задачах виявлення шахрайства важливо мінімізувати false negatives).

Для оцінки моделі можемо скористатися рядом метрик: Accuracy, Precision, Recall, F1-score (для кожного класу), Confusion Matrix, AUC-ROC (для мультикласу — one-vs-rest). Оскільки дані розподілені нерівномірно, то варто також звернути увагу на *зважені метрики* або *балансування класів* (наприклад, SMOTE).

Відповідно, для нашої моделі обираємо:
- **F1-score**: середнє між precision і recall, що важливо для мультикласових задач;
- **AUC-ROC**: показує, наскільки добре модель відрізняє один клас від усіх інших. Для мультикласу використовується підхід *One-vs-Rest* (обчислюється AUC для кожного класу окремо, а потім береться середнє).
- **Confusion Matrix**: показує, де модель помиляється: які класи плутає, які передбачає правильно.

## Завдання 4
Оберіть спосіб поділу даних на навчальні та тестові. Використовуйте класичний train_test_split або, за необхідності, TimeSeriesSplit чи StratifiedKFold. Поясніть логіку: наприклад, «оскільки дані мають часову природу — використовуємо розділення по часовій осі».

Оскільки дані не мають *часового компонента*, але класи *нерівномірні*, ми будемо використовувати **StratifiedKFold**, щоб зберегти пропорції класів у кожному фолді та уникнути перекосу в оцінці моделі.

## Завдання 5
Реалізуйте тренування моделей на ваших train-даних (отриманих з БД). Результати тренування мають логуватись у таблицю predictions (source="train").

In [33]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('./db/obesity_data_processed.db')

q = "SELECT * FROM obesity_data_processed"
processed_data = pd.read_sql_query(q, conn)
processed_data["timestamp"] = pd.to_datetime(processed_data["timestamp"])

conn.close()

print(processed_data.head())
print("\nРозподіл класів:\n", processed_data["NObeyesdad"].value_counts())

   Gender       Age    Height    Weight  family_history_with_overweight  FAVC  \
0     0.0  0.148936  0.320755  0.186567                             1.0   0.0   
1     0.0  0.148936  0.132075  0.126866                             1.0   0.0   
2     1.0  0.191489  0.660377  0.283582                             1.0   0.0   
3     1.0  0.276596  0.660377  0.358209                             0.0   0.0   
4     1.0  0.170213  0.622642  0.379104                             0.0   0.0   

   FCVC       NCP      CAEC  SMOKE  CH2O  SCC       FAF  TUE      CALC  \
0   0.5  0.666667  0.666667    0.0   0.5  0.0  0.000000  0.5  1.000000   
1   1.0  0.666667  0.666667    1.0   1.0  1.0  1.000000  0.0  0.666667   
2   0.5  0.666667  0.666667    0.0   0.5  0.0  0.666667  0.5  0.333333   
3   1.0  0.666667  0.666667    0.0   0.5  0.0  0.666667  0.0  0.333333   
4   0.5  0.000000  0.666667    0.0   0.5  0.0  0.000000  0.0  0.666667   

   MTRANS  NObeyesdad           timestamp  
0    0.75    0.166667 20

In [34]:
from sklearn.model_selection import train_test_split

x = processed_data.drop("NObeyesdad", axis=1)
y = processed_data["NObeyesdad"]

x_train, x_test, y_train, y_test = train_test_split(
  x, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Тренувальний набір: {x_train.shape}, Тестовий набір: {x_test.shape}")

print("\nРозподіл класів у тренувальному наборі:\n", y_train.value_counts(normalize=True))
print("\nРозподіл класів у тестовому наборі:\n", y_test.value_counts(normalize=True))

Тренувальний набір: (1688, 17), Тестовий набір: (423, 17)

Розподіл класів у тренувальному наборі:
 NObeyesdad
0.333333    0.166469
0.666667    0.153436
0.500000    0.140403
0.833333    0.137441
1.000000    0.137441
0.166667    0.135664
0.000000    0.129147
Name: proportion, dtype: float64

Розподіл класів у тестовому наборі:
 NObeyesdad
0.333333    0.165485
0.666667    0.153664
0.500000    0.141844
0.166667    0.137116
1.000000    0.137116
0.833333    0.137116
0.000000    0.127660
Name: proportion, dtype: float64


In [35]:
# Додаємо цільову змінну до тренувального та тестового наборів
train_data = x_train.copy()
train_data["NObeyesdad"] = y_train

test_data = x_test.copy()
test_data["NObeyesdad"] = y_test

train_data.to_csv("./data/train_data.csv", index=False)
test_data.to_csv("./data/test_data.csv", index=False)


In [36]:
conn = sqlite3.connect("./db/split_data.db")

train_data.to_sql("train_data", conn, if_exists="replace", index=False)

test_data.to_sql("test_data", conn, if_exists="replace", index=False)

conn.close()


In [37]:
print("Тренувальний набір:\n", y_train.value_counts(normalize=True))
print("\nТестовий набір:\n", y_test.value_counts(normalize=True))

Тренувальний набір:
 NObeyesdad
0.333333    0.166469
0.666667    0.153436
0.500000    0.140403
0.833333    0.137441
1.000000    0.137441
0.166667    0.135664
0.000000    0.129147
Name: proportion, dtype: float64

Тестовий набір:
 NObeyesdad
0.333333    0.165485
0.666667    0.153664
0.500000    0.141844
0.166667    0.137116
1.000000    0.137116
0.833333    0.137116
0.000000    0.127660
Name: proportion, dtype: float64


In [38]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

model = CatBoostClassifier(verbose=0, random_state=42)
model.fit(x_train, y_train)

import sqlite3
import pandas as pd
from datetime import datetime

y_pred_train = model.predict(x_train)

# Формуємо DataFrame для логування
log_df = pd.DataFrame({
    "timestamp": [datetime.now().isoformat()] * len(y_pred_train),
    "source": ["train"] * len(y_pred_train),
    "actual": y_train,
    "predicted": y_pred_train
})

# Запис у базу даних obesity_data_processed.db
conn = sqlite3.connect("obesity_data_processed.db")
log_df.to_sql("predictions", conn, if_exists="append", index=False)
conn.close()

print("✅ Результати тренування залоговано у таблицю 'predictions'.")


ValueError: Per-column arrays must each be 1-dimensional